In [ ]:
!pip install accelerate -U

In [2]:
!pip install -q transformers datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 22.3 MB/s eta 0:00:00


In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer
import torch, os
import pandas as pd
from torch.utils.data import Dataset

In [6]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
df_org = pd.read_csv("train.csv")
df_org.rename(columns={'labels': 'category','tweets':'text'},inplace=True)
df_org.head(10)

,text,category
0,sending solidarity whoever doctor manage incre...,Stressed
1,need see hair amp beard gat book appointment b...,Anxious
2,next time meet someone new dont ask ask love,Normal
3,surprise someone love give la senza gift box r...,Lonely
4,raise hand junhoes ocean lotion life rent free...,Normal
5,mariposa de barrio teach matter guy forever ch...,Normal
6,good point remember 1013 leave alone pokie clu...,Anxious
7,mori wip white dress,Normal
8,okay ik lot people go want im make gc ashnikko...,Anxious
9,im gonna say,Normal


In [8]:
labels = df_org['category'].unique().tolist()
labels = [s.strip() for s in labels ]
labels

['Stressed', 'Anxious', 'Normal', 'Lonely']

In [9]:
for key, value in enumerate(labels):
    print(value)

Stressed
Anxious
Normal
Lonely


In [10]:
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [11]:
df_org["labels"]=df_org.category.map(lambda x: label2id[x.strip()])
df_org.head(5)

,text,category,labels
0,sending solidarity whoever doctor manage incre...,Stressed,0
1,need see hair amp beard gat book appointment b...,Anxious,1
2,next time meet someone new dont ask ask love,Normal,2
3,surprise someone love give la senza gift box r...,Lonely,3
4,raise hand junhoes ocean lotion life rent free...,Normal,2


In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    "marcus2000/model-mental-health-classification-1e-5-big",
    num_labels=NUM_LABELS, id2label=id2label, label2id=label2id,
    ignore_mismatched_sizes=True)

model.to(device)
tokenizer = AutoTokenizer.from_pretrained("marcus2000/model-mental-health-classification-1e-5-big", max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at marcus2000/model-mental-health-classification-1e-5-big and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [9]:
SIZE= df_org.shape[0]

train_texts= list(df_org.text[:SIZE//2])

val_texts=   list(df_org.text[SIZE//2:(3*SIZE)//4 ])

test_texts=  list(df_org.text[(3*SIZE)//4:])

train_labels= list(df_org.labels[:SIZE//2])

val_labels=   list(df_org.labels[SIZE//2:(3*SIZE)//4])

test_labels=  list(df_org.labels[(3*SIZE)//4:])

In [10]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [11]:
class DataLoader(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

In [13]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)

    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }


In [14]:
import os
os.mkdir('Pre_Train_models')

In [15]:
training_args = TrainingArguments(
    output_dir='/content/',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=10,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='/content/',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    fp16=True
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics= compute_metrics
)

In [18]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.529800,0.656395,0.699653,0.687906,0.739553,0.718734
200,0.521200,0.630330,0.706455,0.694191,0.732338,0.723250
300,0.494600,0.693047,0.701787,0.616846,0.573165,0.689352
400,0.517200,0.631891,0.700187,0.627194,0.689861,0.690638
500,0.463900,0.607463,0.708322,0.694575,0.735589,0.726346
600,0.502700,0.630150,0.706588,0.624119,0.727282,0.693847
700,0.435100,0.671896,0.701520,0.623627,0.634944,0.690337
800,0.516300,0.609734,0.708589,0.654530,0.690704,0.699015
900,0.533700,0.622162,0.703254,0.622907,0.654230,0.690198
1000,0.486000,0.630147,0.708856,0.626545,0.686418,0.696891


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=2814, training_loss=0.4549963560694062, metrics={'train_runtime': 688.037, 'train_samples_per_second': 65.386, 'train_steps_per_second': 4.09, 'total_flos': 1641466294457184.0, 'train_loss': 0.4549963560694062, 'epoch': 3.0})

In [19]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.358871,0.784943,0.778104,0.815536,0.798235
val,0.652836,0.706588,0.702982,0.724378,0.722121
test,0.643619,0.702454,0.700258,0.730599,0.725443


## Save The Model

In [20]:
model_path = "mental-health-tweets-classification-model-2"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('mental-health-tweets-classification-model-2/tokenizer_config.json',
 'mental-health-tweets-classification-model-2/special_tokens_map.json',
 'mental-health-tweets-classification-model-2/vocab.txt',
 'mental-health-tweets-classification-model-2/added_tokens.json',
 'mental-health-tweets-classification-model-2/tokenizer.json')

## Load the Model

In [21]:
tokenizer = AutoTokenizer.from_pretrained("mental-health-tweets-classification-model-2")
model = AutoModelForSequenceClassification.from_pretrained("mental-health-tweets-classification-model-2")
nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

### testing some tweets

In [22]:
df_org.head(5)
stressed_example = df_org['text'][0]
normal_example  = df_org['text'][2]
lonely_example = df_org['text'][3]
anxious_example = df_org['text'][1]
df_org.head(5)

,text,category,labels
0,sending solidarity whoever doctor manage incre...,Stressed,0
1,need see hair amp beard gat book appointment b...,Anxious,1
2,next time meet someone new dont ask ask love,Normal,2
3,surprise someone love give la senza gift box r...,Lonely,3
4,raise hand junhoes ocean lotion life rent free...,Normal,2


In [23]:
nlp('your help is needed')

[{'label': 'Anxious', 'score': 0.6967431902885437}]